# Advent of Code 2024 - J09

In [1]:
def read_input(kind):
    assert kind in ('input', 'example'), '"kind" must be "input" or "example"'
    with open(kind) as f:
        for line in f:
            return line.strip()

In [2]:
edisk_map = read_input("example")
idisk_map = read_input("input")
edisk_map

'2333133121414131402'

## First part

In [3]:
def get_filesystem(disk_map):
    is_file = True
    id_number = 0
    filesystem = []
    for space in disk_map:
        for _ in range(int(space)):
            if is_file:
                filesystem.append(str(id_number))
            else:
                filesystem.append('.')
        if is_file:
            id_number += 1
        is_file = not(is_file)
    return filesystem

In [4]:
get_filesystem("12345")

['0', '.', '.', '1', '1', '1', '.', '.', '.', '.', '2', '2', '2', '2', '2']

In [5]:
def first_part(kind):
    disk_map = read_input(kind)
    filesystem = get_filesystem(disk_map)
    checksum = 0
    index_end = len(filesystem) - 1
    for i, car in enumerate(filesystem):
        if i > index_end:
            print(f"i: {i}, index_end: {index_end}")
            return checksum
        if car == '.':
            while filesystem[index_end] == '.':
                index_end -= 1
                if i >= index_end:
                    print(f"i: {i}, index_end: {index_end}")
                    return checksum
            checksum += int(filesystem[index_end]) * i
            index_end -= 1
        else:
            checksum += int(car) * i
    return checksum

In [6]:
first_part("example")

i: 28, index_end: 27


1928

In [7]:
first_part("input")

i: 50126, index_end: 50125


6344673854800

## Second part

In [8]:
def get_filesystem_by_blocks(disk_map):
    is_file = True
    id_number = 0
    blockfilesystem = []
    for space in disk_map:
        if is_file:
            blockfilesystem.append({
                "is_file": is_file,
                "space": int(space),
                "id_number": id_number
            })
            id_number += 1
        else:
            blockfilesystem.append({
                "is_file": is_file,
                "space": int(space),
                "id_number": None
            })
        is_file = not is_file
    return blockfilesystem

In [9]:
get_filesystem_by_blocks(edisk_map)

[{'is_file': True, 'space': 2, 'id_number': 0},
 {'is_file': False, 'space': 3, 'id_number': None},
 {'is_file': True, 'space': 3, 'id_number': 1},
 {'is_file': False, 'space': 3, 'id_number': None},
 {'is_file': True, 'space': 1, 'id_number': 2},
 {'is_file': False, 'space': 3, 'id_number': None},
 {'is_file': True, 'space': 3, 'id_number': 3},
 {'is_file': False, 'space': 1, 'id_number': None},
 {'is_file': True, 'space': 2, 'id_number': 4},
 {'is_file': False, 'space': 1, 'id_number': None},
 {'is_file': True, 'space': 4, 'id_number': 5},
 {'is_file': False, 'space': 1, 'id_number': None},
 {'is_file': True, 'space': 4, 'id_number': 6},
 {'is_file': False, 'space': 1, 'id_number': None},
 {'is_file': True, 'space': 3, 'id_number': 7},
 {'is_file': False, 'space': 1, 'id_number': None},
 {'is_file': True, 'space': 4, 'id_number': 8},
 {'is_file': False, 'space': 0, 'id_number': None},
 {'is_file': True, 'space': 2, 'id_number': 9}]

In [10]:
def filesystem_by_blocks_to_list(filesystem):
    s = []
    for block in filesystem:
        if block["is_file"]:
            for _ in range(block["space"]):
                s.append(str(block["id_number"]))
        else:
            for _ in range(block["space"]):
                s.append('.')
    return s

In [11]:
print(filesystem_by_blocks_to_list(get_filesystem_by_blocks(edisk_map)))

['0', '0', '.', '.', '.', '1', '1', '1', '.', '.', '.', '2', '.', '.', '.', '3', '3', '3', '.', '4', '4', '.', '5', '5', '5', '5', '.', '6', '6', '6', '6', '.', '7', '7', '7', '.', '8', '8', '8', '8', '9', '9']


In [12]:
def reorder_blocks(filesystem):
    index_end = len(filesystem) - 1
    while index_end > 0:
        if filesystem[index_end]["is_file"]:
            for i in range(index_end):
                if (not filesystem[i]["is_file"]) and (filesystem[i]["space"] >= filesystem[index_end]["space"]): # Look for span of free space large enough
                    # Move block and adjust size of remaining free space
                    size_moving_block = filesystem[index_end]["space"]
                    remaining_free_space = filesystem[i]["space"] - size_moving_block
                    filesystem[i]["is_file"] = True
                    filesystem[i]["space"] = filesystem[index_end]["space"]
                    filesystem[i]["id_number"] = filesystem[index_end]["id_number"]
                    if remaining_free_space > 0:
                        filesystem.insert(i+1, {'is_file': False, 'space': remaining_free_space, 'id_number': None})
                        index_end += 1
                    # Replace old block by free space
                    filesystem[index_end]["is_file"] = False
                    filesystem[index_end]["id_number"] = None
                    # And out of this loop
                    break
        index_end -= 1
    return filesystem

In [13]:
def second_part(kind):
    disk_map = read_input(kind)
    filesystem = get_filesystem_by_blocks(disk_map)
    reordered_filesystem = reorder_blocks(filesystem)
    f = filesystem_by_blocks_to_list(reordered_filesystem)
    assert len(filesystem_by_blocks_to_list(filesystem)) == len(f), "Filesystem should be the same length after reorder than before!"
    checksum = 0
    for i, car in enumerate(f):
        if car != '.':
            checksum += int(car) * i
    return checksum

In [14]:
second_part("example")

2858

In [15]:
second_part("input")

6360363199987